<img src="images/strathsdr_banner.png" align="left">

# RFSoC Modulation Classification

## Overview
> This work presents a real-time FPGA-based Convolutional Neural Network (CNN) architecture for modulation classification, implemented on the Zynq UltraScale+ RFSoC2x2 development board. Leveraging General Matrix Multiplication (GEMM) optimisations, fixed-point arithmetic, and quantisation-aware training, we develop models with 16-bit, 8-bit, and 4-bit weight precisions. A novel method for generating wireless communication datasets using the RFSoC is also introduced. The design integrates with the open-source PYNQ framework, enabling real-time modulation prediction via an interactive Jupyter app. The 16-bit model demonstrates superior accuracy with efficient resource use, highlighting its suitability for receiver deployment. The demo showcases the architecture's performance and practical application in real-world scenarios.

<figure>
<img src='./images/experimentalSetup.svg' width='80%'/>
<figcaption><b>Figure 1: The architecture overview of the RFSoC classifying signals.</b></figcaption>
</figure>

### CNN Models
The CNNs that are presented all include the same dimensions of layers, as seen in Table 1. The only difference exists between the precision of the weights and kernels of each model (16-bit, 8-bit, and 4-bit). The precision of the interlayer activations remains at 16-bits.
<figure>
<img src='./images/nndimensions.png' width='30%'/>
<figcaption><b>Table 1: Neural Network Dimensions</b></figcaption>
</figure>

### Streaming-CNN FPGA architecture for Radio receiver applications
This work demonstrates the CNN architecture presented by Maclellan et al. [2][3]. The architecture aims to process samples received by the ADC in real-time and continuously without dropping samples. The topological optimisations were chosen based on the flow of the received data throughout the CNN model.

## Aims
- Introduce the problem of modulation classification
- Select modulation schemes and transmit via RF-DAC.
- Receive signal via RF-ADC and classify modulation scheme.

## Last Revised
- 08/11/24 - Initial revision
----

## Hardware Setup
For this demonstration, the RFSoC2x2 development board should be connected in RF loopback, with DAC A connected to ADC A. In line with the RF loopback cable insert a [Nooelec VeGA](https://www.nooelec.com/store/vega-barebones.html) variable gain amplifier and low-pass filter. The VeGA should be configured to digital mode with switches D0-5 in the off position.

<figure>
<img src='./images/board_loopback.png' width='500%'/>
<figcaption><b>Figure 2: RF loopback with VeGA and Low-Pass Filter.</b></figcaption>
</figure>

<div class="alert alert-heading alert-danger">
<b>Warning:</b> The VeGA switches should be configured in digital mode with switches D0-5 to the off position. Changing these settings may provide too much power and saturate the ADC.
</div>

## Data set
DeepSig.ai's RadioML data set [1] quickly established itself as a benchmark for evaluating DL architectures within the realm of wireless communications. There exists a disparity between the RadioML data set and the actual input received by the ADC of the RFSoC. This difference arises from the absence of additional factors that can deteriorate the signals impacted by the transceiver itself, such as the effects of applying digital filters that do not have an 'ideal' brick wall response, spurs, and harmonics introduced by the data converter, the influence of the ADC quantisation and signal power, and the possible artifacts associated with the stitching of the interleaved ADCs. Additionally, the RadioML data is provided in fixed length frames of 128 complex samples, yet for real-time classification a continuous data stream is required so that it can be output from the DAC.

The data set created for this application uses the RFSoC within the generation loop. This is a pivotal step towards creating a data set that closely represents how data passes through the RFSoC, including the analogue and RF processing stages, and ultimately developing a DL network trained with this context in mind.

<figure>
<img src='./images/dataset_generation.png' width='80%'/>
<figcaption><b>Figure 3: Data set generation process.</b></figcaption>
</figure>

The resulting data set consists of 8 modulation schemes:
- QPSK
- BPSK
- QAM16
- QAM64
- PSK8
- PAM4
- GFSK
- CPFSK

With additive white Gaussian noise of -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20, 24, 28, and 30dB SNR.

### Inspect the data set prior to DAC transmission

In [ ]:
from rfsoc_quant_amc import helper as h

filename = './transmit_test_SNR.pkl'
dataset = h.load_dataset(filename)

# Inspect an example
mod = 'QPSK'
snr = '30'
data_mod = dataset[mod,snr][:,:,0]
h.plot_dataset(data_mod)

## Setup RF Data Converters
DAC-A and ADC-A need to be configured in order to successfully transmit and receive signals.

## Load Overlay
Load the overlay through the custom `Overlay` class.

In [ ]:
from rfsoc_quant_amc.overlay import Overlay

ol = Overlay()

### RF-ADC
First the RF-ADC channel is configured.

In [ ]:
ADC_SAMPLING_FREQ = 1024.0 # MHz
ADC_PLL_FREQ = 409.6 # MHz
centre_freq = 400 # MHz

ol.initialise_adcs(pll_freq=ADC_PLL_FREQ, 
                   sampling_freq=ADC_SAMPLING_FREQ, 
                   centre_freq=centre_freq)

### RF-DAC
Next, the RF-DAC channel is configured.

In [ ]:
DAC_SAMPLING_FREQ = 1024.0 # MHz
DAC_PLL_FREQ = 409.6 # MHz
centre_freq = 400 # MHz

ol.initialise_dacs(pll_freq=DAC_PLL_FREQ, 
                   sampling_freq=DAC_SAMPLING_FREQ, 
                   centre_freq=centre_freq)

In [ ]:
ol.initialise_ips()

## Transmit data through DAC
The data is transmitted from PS via an AXI Direct Memory Access IP configured in `cyclic` mode. The selected signal from the data set, which is 4,096 complex samples long, is `allocate` into memory and repeatedly transmitted. The signal passes through a chain of FIR interpolation filters increasing the sample rate of the signal to 128MHz. Finally, a carrier is applied to the signal prior to it being transmitted.

In [ ]:
import numpy as np
mods = ['QPSK','BPSK','QAM16','QAM64','PSK8','PAM4','GFSK','CPFSK']
snrs = ['-20', '-16', '-12', '-8', '-4', '0', '4', '8', '12', '16', '20', '24', '28','30']

mod = 'QPSK' # Choose from the selection of modulation schemes
snr = '30'   # Choose from the selection of SNRs
data_mod = dataset[mod,snr][:,:,0]
# Scale to fit DAC range
y = np.int16(data_mod*np.int16(pow(2,13)))
# Interleave complex samples
z = np.zeros(2*4096, dtype=np.int16)
z[0::2] = y[0,:]
z[1::2] = y[1,:]
# Send to DAC
ol.send(z)

## Receive data through ADC
The data is received at the ADC and passed through a chain of FIR decimation filters. The signal is then passed into each of the AMC models where each prediction is sent to the PS through an AXI-DMA. Additionally, the received signal is also packetised and sent to the PS.

In [ ]:
[y_pred_0, y_pred_1, y_pred_2, re_data, im_data] = ol.receive_data()
print(f"Model 16w16a predicted {mods[y_pred_0]}")
print(f"Model  8w16a predicted {mods[y_pred_1]}")
print(f"Model  4w16a predicted {mods[y_pred_2]}")

### Inspect received signal
The above models predicted the modulation scheme for the received signal. The following figure plots the samples that were given to the CNN models.

In [ ]:
h.plot_received(re_data, im_data)

## Interactive App
The following cells run the `ipywidgets` app. The app provides an interative experience when experimenting with the CNN models.

### Initialise the app class
The `AMCWidget` class takes the overlay as an argument as well as the dataset, mods, and snrs.

The app allows the user to choose the modulation scheme, noise, and phase offset. The 'Update' button reloads the data in the cyclic DMA and the 'play' button continually initialises the DMA transfer to read the CNN predictions.

In [ ]:
from rfsoc_quant_amc.widget import AMCWidget
widget = AMCWidget(ol, dataset, mods, snrs)
widget.display()

## Overall Accuracy
The model's performance is evaluated by each quantised model's ability to predict modulation schemes across various noise levels, as shown in the plots below. The 16-bit model shows poor performance at low SNR but achieves 80-100% accuracy for most modulation schemes at higher SNRs, although struggling to differentiate the QAM16 and QAM64 modulation schemes.

The 8-bit weight model shows similar performance but due to the reduction in weight size, it loses accuracy. This is further exaggerated with the 4-bit weight model where a significant drop in accuracy is seen.

In [ ]:
from rfsoc_quant_amc.helper import load_results, plot_accuracies

plot_16w, plot_8w, plot_4w, acc_16w, acc_8w, acc_4w = load_results()

In [ ]:
fig = plot_accuracies(acc_16w, acc_8w, acc_4w)
fig.show()

### 16-bit weight model confusion matrices
The following confusion matrices showcase the performance of the 16w16a CNN model at a selection of SNRs from -16, -8, 0, 8, 16, 24, 28, and 30dB SNR.

In [ ]:
plot_16w.show()

### 8-bit weight model confusion matrices
The following confusion matrices showcase the performance of the 8w16a CNN model at a selection of SNRs from -16, -8, 0, 8, 16, 24, 28, and 30dB SNR.

In [ ]:
plot_8w.show()

### 4-bit weight model confusion matrices
The following confusion matrices showcase the performance of the 4w16a CNN model at a selection of SNRs from -16, -8, 0, 8, 16, 24, 28, and 30dB SNR.

In [ ]:
plot_4w.show()

## References
[1] T. J. O’Shea and J. Corgan, “Convolutional radio modulation recognition networks,” CoRR, vol. abs/1602.04105, 2016. [Online]. Available: http://arxiv.org/abs/1602.04105

[2] A. Maclellan, L. H. Crockett and R. W. Stewart, "Streaming Convolutional Neural Network FPGA Architecture for RFSoC Data Converters," 2023 21st IEEE Interregional NEWCAS Conference (NEWCAS), Edinburgh, United Kingdom, 2023, pp. 1-5, doi: 10.1109/NEWCAS57931.2023.10198198

[3] A. Maclellan, L. H. Crockett and R. W. Stewart, "RFSoC Modulation Classification With Streaming CNN: Data set Generation & Quantized-Aware Training," in IEEE Open Journal of Circuits and Systems, doi: 10.1109/OJCAS.2024.3509627.